In [1]:
# %env MINERL_DATA_ROOT=/Volumes/CORSAIR/data

In [2]:
import minerl  # NOTE: we need gym>=0.13.1,<0.20
import gym
from train import get_agent, get_dynamics_environment, FMC, get_data_handler
from fgz.training.fgz_trainer import FGZTrainer
import torch
import wandb

/home/nollyd/anaconda3/envs/minerl/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.cuda.empty_cache()  # fix memory leaks

In [4]:
minerl_env = gym.make('MineRLBasaltFindCave-v0')
agent = get_agent()
dynamics_env = get_dynamics_environment(minerl_env)

Loading model data/VPT-models/foundation-model-1x.model
with weights data/VPT-models/foundation-model-1x.weights


In [5]:
agent.device

device(type='cuda')

In [6]:
# dynamics_env.batched_action_space_sample()

In [7]:
# dummy_initial_state = torch.ones(4096, dtype=float)
# dynamics_env.set_all_states(dummy_initial_state)

In [8]:
data_handler = get_data_handler(agent)
str(data_handler.loaders[0].trajectories[0])

'T(Player281-4e85ecba5b7a-20220706-205440)'

In [9]:
# t = data_handler.sample_single_trajectory()
# for window in t:
#     print(len(window))
#     for frame, state_embedding, action in window:
#         print(state_embedding.shape)
#         break
#     break

In [10]:
dynamics_function_optimizer = torch.optim.SGD(
    dynamics_env.dynamics_function.parameters(),
    lr=0.01,
)

In [11]:
use_wandb = True

fmc = FMC(dynamics_env, freeze_best=True)
trainer = FGZTrainer(minerl_env, agent, fmc, data_handler, dynamics_function_optimizer, unroll_steps=8, use_wandb=use_wandb)

if use_wandb:
    wandb.init(project="fgz-v0")

2022-10-09 20:02:57 nollyd wandb.jupyter[743238] ERROR Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
2022-10-09 20:02:58 nollyd urllib3.connectionpool[743238] DEBUG Starting new HTTPS connection (1): api.wandb.ai:443
2022-10-09 20:02:58 nollyd urllib3.connectionpool[743238] DEBUG https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None
2022-10-09 20:02:58 nollyd urllib3.connectionpool[743238] DEBUG Starting new HTTPS connection (1): api.wandb.ai:443
2022-10-09 20:02:58 nollyd urllib3.connectionpool[743238] DEBUG https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None
wandb: Currently logged in as: dyllan. Use `wandb login --relogin` to force relogin
2022-10-09 20:02:58 nollyd git.cmd[743238] DEBUG Popen(['git', 'cat-file', '--batch-check'], cwd=/home/nollyd/Code/basalt_2022_competition_nollied, universal_newlines=False, shell=None, istream=<valid stream>)


In [12]:
trainer.train_sub_trajectory(use_tqdm=True, max_steps=None)

Training on BuildWaterfall: Player757-f153ac423f61-20220704-235445:   0%|          | 0/8 [00:00<?, ?it/s]/home/nollyd/anaconda3/envs/minerl/lib/python3.10/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
Training on BuildWaterfall: Player757-f153ac423f61-20220704-235445: 100%|██████████| 8/8 [00:01<00:00,  5.35it/s]


In [13]:
# fmc.simulate(4, use_tqdm=True)

In [14]:
# fmc.tree.render() ;

In [15]:
# for state, action in fmc.tree.best_path:
#     print(state.reward, state.observation.var(), action)
#     print()

In [16]:
# trainer.train_trajectory()